# Proteína ERK2
### Obtención de los ligandos ubicados en la cavidad de la proteína
Esta libreta tinene como objetivo obtener los **ligandos cocristalizados en ele sitio activo de la proteína ERK2**, difereciando dichas moléculas de iones o moléculas del solvente, así como de ligandos o inhibidores alostéricos.

In [1]:
from prody import *
import numpy as np
import csv
import pandas as pd

In [4]:
# Nombre de la proteina de interes
prot_name = 'erk2'

**Carga del dataframe con la información de la proteína**

In [89]:
# abrir la tabla de metadatos de las proteinas
import glob, os
# Directorio hacia el archivo csv
path_df_csv = glob.glob(os.path.join('.', 'data', 
                                     F'TABLA_MTDATA_{prot_name.upper()}_*_crys.csv'))

df_prot = pd.read_csv(path_df_csv[0], index_col = 0,
                     converters={"NameLigs": lambda x: x.strip("[]").replace("'", "").split(", ")}, 
                      # Convierte en lista la celda del nombre de los ligandos
                      # A falta de una solución más prolija
                      # TODO: Trabajar con archivos json
                     )
df_prot.set_index('PDB_ID', inplace=True)
df_prot.tail(3)

,Title,Date,Entities,ChainID,Resolution,Identity,Coverage,NumGaps,GapLen,GapPos,NumLigs,NameLigs
PDB_ID,,,,,,,,,,,,
6qal,erk2 mini-fragment binding,2019-03-27,1,A,1.57,359.0,93.055556,5.0,"[10, 3, 1, 8, 3]","[[1, 10], [34, 36], [161, 161], [331, 338], [3...",3.0,"[CME, HV2, SO4]"
6qaq,erk2 mini-fragment binding,2019-03-27,1,A,1.58,359.0,95.277778,4.0,"[10, 1, 3, 3]","[[1, 10], [161, 161], [331, 333], [358, 360]]",3.0,"[CME, HVQ, SO4]"
6qaw,erk2 mini-fragment binding,2019-03-27,1,A,1.84,358.0,93.333333,5.0,"[10, 1, 1, 9, 3]","[[1, 10], [127, 127], [161, 161], [330, 338], ...",3.0,"[CME, HVE, SO4]"


In [85]:
df_prot.shape

(166, 12)

### Conteo de estructuras con ligandos
Preguntamos cuantas y cuales estructuras cristalográficas tienen ligandos y creamos una nueva tabla con ellas.  

In [91]:
df_prot_ligs = df_prot[df_prot.NumLigs > 0]
print(F'{len(df_prot_ligs)} de {len(df_prot)} cristales con al menos un ligando.')

156 de 166 cristales con al menos un ligando.


**No obstante, no todas ellas son ligandos de la proteína**, ya que sólo hemos identificado cadenas en el archivo pdb con la designación **HETATM**. Si es ligando o no lo determinamos más adelante.

### ¿Cuántos ligandos diferentes hay?
Ahora preguntamos cuántas moléculas distintas han sido cocristalizadas con la proteína.

In [92]:
# ¿Cuantos ligandos hay en total?
import itertools
# Lista de nombre de todos los ligandos
lig_full_list = list( itertools.chain.from_iterable( df_prot.NameLigs ) )
print(F'En total hay {len(lig_full_list)} moléculas HETATM,')


lig_all_list = list(set(lig_full_list))
print(F'de las cuales {len(lig_all_list)} son moléculas DISTINTAS.')

En total hay 459 moléculas HETATM,
de las cuales 152 son moléculas DISTINTAS.


## Moléculas en la cavidad catalítica
Ahora procedemos a identificar qué moléculas se encuentran en el sitio activo de la proteína, y si éstas tienen un peso molecular suficcientemente grande como para ser diferenciadas de posibles moléculas del solvente.

### Primero definimos qué residuos forman parte del sitio activo
Tomamos como referencia las primeras dos estructuras del DataFrame con ligandos:

In [93]:
df_prot_ligs.head(2)

,Title,Date,Entities,ChainID,Resolution,Identity,Coverage,NumGaps,GapLen,GapPos,NumLigs,NameLigs
PDB_ID,,,,,,,,,,,,
1gol,coordinates of rat map kinase erk2 with an arg...,1997-03-12,1,A,2.8,354.0,99.166667,1.0,[3],"[[1, 3]]",2.0,"[ATP, MG]"
1pme,structure of penta mutant human erk2 map kinas...,1999-06-08,1,A,2.0,354.0,92.500000,4.0,"[15, 3, 1, 8]","[[1, 15], [33, 35], [161, 161], [330, 337]]",3.0,"[CME, SB2, SO4]"


In [61]:
print(F'Estructuras de referencia:\n{df_prot_ligs.index[:2]}')
second_ref_name = '1pme'

Estructuras de referencia:
Index(['1gol', '1pme'], dtype='object', name='PDB_ID')


In [95]:
df_prot_ligs.loc[first_ref_name].NameLigs[0]

'ATP'

In [104]:
cutoff = 7
first_ref_name = '1gol'
ligand_name = df_prot_ligs.loc[first_ref_name].NameLigs[0]

# Obtenemos la estructura
first_ref_strc = parsePDB(first_ref_name)
# Selecion de los residuos a 5 agstroms del ligando en la cavidad
selected_residues = first_ref_strc.select(F'within {cutoff} of resname {ligand_name} and name CA')
# Obtenemos la secuencia de los residuos
selected_residues.getResindices()

@> PDB file is found in working directory (1gol.pdb.gz).
@> 3012 atoms and 1 coordinate set(s) were parsed in 0.04s.


array([ 27,  28,  29,  30,  35,  47,  48,  49,  50, 101, 102, 103, 104,
       105, 106, 107, 149, 150, 152, 163])

Esta selección debe corroborarse de forma visual, asegurándonos que la molécula sea un inhibidor a sustrato en el sitio de interés de la proteína. Pra ello se puede usar UCSF Chimera, o, en este caso nglviewer:

In [150]:
selected_residues.ge

<Selection: 'within 7 of res...ATP and name CA' from 1gol (20 atoms)>

In [149]:
import nglview as nv
view = nv.show_prody(first_ref_strc)
view.clear_representations()
view.add_representation('cartoon', selection='protein', color='gray',
                       opacity=0.1)
view.add_representation('licorice',
                       selection = 'ATP',
                       color='red')
view

NGLWidget()

In [6]:
from Funciones.subsecuencias_cdk2 import get_ricci_residues
ricci_residues = get_ricci_residues('str')
print(len(get_ricci_residues('list')), 
      "residuos corresponden a los que definen al pocket de la proteína CDK2.")
print(ricci_residues)

40 residuos corresponden a los que definen al pocket de la proteína CDK2.
8 9 10 11 12 13 14 15 16 17 18 19 30 31 32 33 64 65 79 80 81 82 83 84 85 86 87 88 89 90 129 130 131 132 133 134 143 144 145 146


In [15]:
# Estructura de referencia con hidrógenos, la obtenemos del ensamble
pdbs_model_dir = "../ARCHIVOS/CRISTALES/PROT_CDK2_PREPARADAS/PREP_ALL_ENSAMBLE/"
ref_id = '1fin'
pdb_file_tail = "_ENS.pdb"
# Cargamos el archivo de referencia
ref_structure = parsePDB(pdbs_model_dir + ref_id + pdb_file_tail)
ref_structure.setChids("A")

@> 4848 atoms and 1 coordinate set(s) were parsed in 0.08s.


In [16]:
# COMENTADO PORQUE EVITO CONFUNDIRME CON MÄS ESTRUCTURAS: moveAtoms(ref_structure.select('ca resnum ' + ricci_residues ), to=np.zeros(3))
# Cálculo del centro total de la proteína de referencia
centro_strc_total = calcCenter(ref_structure).round(3)
# Definimos los átomos del pocket
pocket = ref_structure.select("ca resnum " + ricci_residues)
centro_pocket = calcCenter(pocket)
print("Centro de la estructura completa:", centro_strc_total)
print("Centro de los Ca del pocket:", centro_pocket)

Centro de la estructura completa: [-20.114 214.997 119.258]
Centro de los Ca del pocket: [-12.5263   206.416975 113.78675 ]


In [29]:
def get_pocket_ligand(pdb_id, ref_structure,
                         PROT_DIR_in = pdbs_model_dir, 
                         RAW_LIG_DIR= "../ARCHIVOS/CRISTALES/LIGS_CDK2/RAW/",
                         #PROT_DIR_out = "../PDB_CDK2_DOCKING/CDK2_PROT_ALIGNED",
                         PREP_LIG_DIR_out= "../ARCHIVOS/CRISTALES/LIGS_CDK2/IN_POCKET_NON_PREP/",
                         zona = 2.0, tail_file_pdb_prot = "_ENS.pdb",
                         pocket_residues = get_ricci_residues(),
                         write_files = True):

    # Se carga el ligando en CRUDO usando Prody
    try:
        lig = parsePDB(RAW_LIG_DIR + "/" + pdb_id  + "_lig.pdb")
        # print("Número de ligandos", len( np.unique(lig.getResnums() )) )
    except Exception as e:
        print(e, "La proteina", pdb_id, "NO TIENE LIGANDO.")
        return(None, None)

    # Se carga la proteína usando los residuos que definen el pocket de la proteína
    protein = parsePDB(PROT_DIR_in + "/" + pdb_id + tail_file_pdb_prot)
    protein_pocket = protein.select("resid " + pocket_residues)

    # Se seleccionan los ligandos (RESIDUOS no protéicos) que estén a no más de X A
    # de cualquier átomo de los residuos del pocket de la proteína
    lig_sel = lig.select('within ' + str(zona) + ' of inhibitor', inhibitor = protein_pocket)
    if lig_sel is None:
        print("La proteina", pdb_id, "no tiene ligando en el pocket.")
        return(None, None)

    # Se obtine la lista de moléculas que cumplen el criterio anterior
    inhibidor_list = np.unique( lig_sel.getResnums() )
    print(lig_sel.getResnums())
    
    # Calcula los centros geométricos de cada molécula
    prot_pocket_center = calcCenter(protein_pocket).round(3)
    
    # Puede que haya más de una molécula en el pocket (debido a presencia de iones o moléculas pequeñas)
    # Se itera entre cada molecula del ligando, se calcula su masa y se mantiene el ligando con mayor masa 
    masa_lig = 1
    final_lig = ""

    for i in inhibidor_list:
            mol = lig.select("resnum " + str(i))
            masa = mol.getMasses().sum()
            print(masa)
            if masa == masa_lig:
                dist_prot_m1 = calcDistance(prot_pocket_center, calcCenter(mol))
                dist_prot_m2 = calcDistance(prot_pocket_center, calcCenter(final_lig))
                if dist_prot_m1 < dist_prot_m2:
                    masa_lig, final_lig = masa, mol
                    continue
            masa_lig, final_lig = (masa, mol) if masa > masa_lig else (masa_lig, final_lig)
            
    # Se extrae el nombre del ligando
    name_lig = np.unique( final_lig.getResnames() )[0]
    # print(name_lig, " es el ligando, con una masa de ", masa_lig)
    
    # Se combinan proteína y ligando en una sola instancia, de tal manera que
    # sea posible superponer las estructuras de la proteína de acuerdo a los residuos de su pocket
    # y esto afcete la psoición del ligando
    complejo_PL = protein + final_lig.toAtomGroup()

    # Se hace el alineamiento con los residuos de Pisani y la estructura de referencia
    # matchAlign(complejo_PL, ref_structure, tarsel= "ca resnum " + ricci_residues())

    # Se guarda el ligando sólo si su masa es igual o mayor a la de una molécula de benceno: 78,11
    # si el ligando aparece 2 veces cerca del pocket, se elige la molécula más cercana al
    # centro geométrico del pocket
    if final_lig.getMasses().sum() >= 70:
        if write_files:
            # Se guardan ligando y proteína alineados según la estructura de referencia
            # Ligando
            writePDB( PREP_LIG_DIR_out+ "/" + pdb_id + "_" + name_lig + "_LIG.pdb", 
                     complejo_PL.select("resname " + name_lig) )
            
           # writePDB( PROT_DIR_out + "/" + pdb_id + "_" + name_lig + "_PROT.pdb", 
           #          complejo_PL.select("protein") )
        return(name_lig, masa_lig)
    
    else: 
        print("La proteina", pdb_id, "NO TIENE LIGANDO (masa => 80.0).")
        return(None, None)

In [30]:
# Se obtienen los ligandos y a la tabla de datos se le agrega la información sobre el inhibidor y su masa

df_cdk2_c = df_cdk2.copy()
# Se agregan dos columnas
df_cdk2_c["Inhib"] = ""
df_cdk2_c["Inhib_mass"] = np.nan
# Se establecen los PDB IDs  como indices
df_cdk2_c = df_cdk2_c.set_index("PDB_ID")

pdb_id_prueba = ["3pxf"]

# Se obtienen los ligandos
for pdb_id in df_cdk2_c.index:
    inhibidor = get_pocket_ligand(pdb_id, write_files=True, 
                                  ref_structure=ref_structure, zona=3.0)
    df_cdk2_c.at[str(pdb_id), 'Inhib'] = inhibidor[0]
    df_cdk2_c.at[pdb_id, 'Inhib_mass'] = inhibidor[1]
    
df_cdk2_c = df_cdk2_c.reset_index()

@> 35 atoms and 1 coordinate set(s) were parsed in 0.00s.
@> 4848 atoms and 1 coordinate set(s) were parsed in 0.17s.
@> 35 atoms and 1 coordinate set(s) were parsed in 0.00s.
@> 4848 atoms and 1 coordinate set(s) were parsed in 0.08s.
@> 35 atoms and 1 coordinate set(s) were parsed in 0.00s.


[299 299 299 299 299 299 299 299 299 299 299 299 299 299 299 299 299]
440.0
[382 381 381 381 381 381 381 381 381 381 381 381 381 381 381 381 381 381
 381 381 381 381 381 381]
398.0
0.0


@> 4848 atoms and 1 coordinate set(s) were parsed in 0.08s.
@> 22 atoms and 1 coordinate set(s) were parsed in 0.00s.
@> 4848 atoms and 1 coordinate set(s) were parsed in 0.07s.
@> 22 atoms and 1 coordinate set(s) were parsed in 0.00s.
@> 4848 atoms and 1 coordinate set(s) were parsed in 0.07s.


[382 381 381 381 381 381 381 381 381 381 381 381 381 381 381 381 381 381
 381 381 381 381 381 381]
398.0
0.0
../ARCHIVOS/CRISTALES/LIGS_CDK2/RAW//1buh_lig.pdb is not a valid filename or a valid PDB identifier. La proteina 1buh NO TIENE LIGANDO.
[299 299 299 299 299 299 299 299 299 299 299 299 300 300 300]
214.0
56.0
[500 500 500 500 500 500 500 500 500 500 500 500 500 500 500]
282.0


@> 35 atoms and 1 coordinate set(s) were parsed in 0.00s.
@> 4848 atoms and 1 coordinate set(s) were parsed in 0.05s.
@> 21 atoms and 1 coordinate set(s) were parsed in 0.00s.
@> 4848 atoms and 1 coordinate set(s) were parsed in 0.05s.
@> 21 atoms and 1 coordinate set(s) were parsed in 0.00s.
@> 4848 atoms and 1 coordinate set(s) were parsed in 0.07s.
@> 24 atoms and 1 coordinate set(s) were parsed in 0.00s.


...

[303 303 303 303 303 303 303 303]
130.0
[301 301 301 301 301 301]
102.0
[301 301 301 301 301 301 301 301 301 301 301 301]
258.0


@> 4848 atoms and 1 coordinate set(s) were parsed in 0.05s.
@> 22 atoms and 1 coordinate set(s) were parsed in 0.00s.
@> 4848 atoms and 1 coordinate set(s) were parsed in 0.05s.
@> 25 atoms and 1 coordinate set(s) were parsed in 0.00s.
@> 4848 atoms and 1 coordinate set(s) were parsed in 0.05s.
@> 46 atoms and 1 coordinate set(s) were parsed in 0.00s.


[301 301 301 301 301 301 301 301 301 301 301 301]
274.0
[302 302 302 302 302 302 302 302 302 302]
234.0
[303 303 303 303 303 303 303 303 303 303 303 303 303]
218.0


@> 4848 atoms and 1 coordinate set(s) were parsed in 0.08s.


[303 303 303 303]
56.0
La proteina 6q4k NO TIENE LIGANDO (masa => 80.0).


In [32]:
df_cdk2_c.to_csv("./B_DATOS/TABLA_MTDATA_CDK2_402_crys_LIGS_INFO.csv")

In [31]:
df_cdk2_c

,PDB_ID,Title,Date,Entities,ChainID,Resolution,Identity,Coverage,NumGaps,GapLen,GapPos,NumLigs,NameLigs,Inhib,Inhib_mass
0,1aq1,human cyclin dependent kinase 2 complexed with...,1997-11-12,1,A,2.00,298.0,92.953020,2.0,"[8, 13]","[[36, 43], [149, 161]]",1.0,['STU'],STU,440.0
1,1b38,human cyclin-dependent kinase 2,1998-12-23,1,A,2.00,298.0,97.315436,1.0,[8],"[[36, 43]]",3.0,"['ACE', 'ATP', 'MG']",ATP,398.0
2,1b39,human cyclin-dependent kinase 2 phosphorylated...,1998-12-23,1,A,2.10,298.0,97.315436,1.0,[8],"[[36, 43]]",3.0,"['ACE', 'ATP', 'MG']",ATP,398.0
3,1buh,crystal structure of the human cdk2 kinase com...,1998-09-09,2,A,2.60,298.0,96.308725,2.0,"[7, 4]","[[40, 46], [295, 298]]",0.0,[],None,NaN
4,1ckp,human cyclin dependent kinase 2 complexed with...,1999-01-13,1,A,2.05,298.0,93.624161,2.0,"[8, 11]","[[36, 43], [153, 163]]",2.0,"['EDO', 'PVB']",PVB,214.0
5,1di8,the structure of cyclin-dependent kinase 2 (cd...,2000-11-29,1,A,2.20,298.0,94.966443,2.0,"[6, 9]","[[37, 42], [153, 161]]",1.0,['DTQ'],DTQ,282.0
6,1dm2,human cyclin-dependent kinase 2 complexed with...,2000-05-31,1,A,2.10,298.0,91.946309,2.0,"[9, 15]","[[36, 44], [149, 163]]",2.0,"['EDO', 'HMD']",HMD,234.0
7,1e1v,human cyclin dependent kinase 2 complexed with...,2001-05-10,1,A,1.95,298.0,97.315436,1.0,[8],"[[36, 43]]",2.0,"['ACE', 'CMG']",CMG,230.0
8,1e1x,human cyclin dependent kinase 2 complexed with...,2001-05-10,1,A,1.85,298.0,97.315436,1.0,[8],"[[36, 43]]",2.0,"['ACE', 'NW1']",NW1,234.0
9,1e9h,thr 160 phosphorylated cdk2 - human cyclin a3 ...,2001-10-11,2,A,2.50,295.0,99.328859,1.0,[2],"[[296, 297]]",2.0,"['INR', 'TPO']",INR,332.0


In [38]:
# Convertir con babel
import os

IN_dir = "./CDK2_INHIBIDORES/"
OUT_dir = "./CDK2_INHIBIDORES_PREPARADOS/"

lista_prueba = ["1qmz"]

# Los ligandos deben convertirse a mol2, debido a que, por alguna razón
# la asignación de cargas no es correcta con archivos pdb
for pdb_id in df_cdk2_c.PDB_ID:
    print(pdb_id)
    os.system("obabel -ipdb " + IN_dir + pdb_id + "*.pdb" + 
              " -omol2 -O " + OUT_dir + pdb_id + "_LIG.mol2" + 
              " -h -p 7")


1aq1
1b38
1b39
1buh
1ckp
1di8
1dm2
1e1v
1e1x
1e9h
1f5q
1fin
1fq1
1fvt
1fvv
1g5s
1gih
1gii
1gij
1gy3
1gz8
1h00
1h01
1h07
1h08
1h0v
1h0w
1h1p
1h1q
1h1r
1h1s
1h24
1h25
1h26
1h27
1h28
1hck
1hcl
1jst
1jsv
1jvp
1ke5
1ke6
1ke7
1ke8
1ke9
1ogu
1oi9
1oiq
1oir
1oit
1oiu
1oiy
1okv
1okw
1ol1
1ol2
1p2a
1p5e
1pf8
1pkd
1pw2
1pxi
1pxj
1pxk
1pxl
1pxm
1pxn
1pxo
1pxp
1pye
1qmz
1r78
1urc
1urw
1v1k
1vyw
1vyz
1w0x
1w8c
1w98
1wcc
1y8y
1y91
1ykr
2a0c
2a4l
2b52
2b53
2b54
2b55
2bhe
2bhh
2bkz
2bpm
2btr
2bts
2c4g
2c5n
2c5o
2c5v
2c5x
2c5y
2c68
2c69
2c6i
2c6k
2c6l
2c6m
2c6o
2c6t
2cch
2cci
2cjm
2clx
2ds1
2duv
2exm
2fvd
2g9x
2i40
2iw6
2iw8
2iw9
2j9m
2jgz
2r3f
2r3g
2r3h
2r3i
2r3j
2r3k
2r3l
2r3m
2r3n
2r3o
2r3p
2r3q
2r3r
2r64
2uue
2uzb
2uzd
2uze
2uzl
2uzn
2uzo
2v0d
2v22
2vta
2vth
2vti
2vtj
2vtl
2vtm
2vtn
2vto
2vtp
2vtq
2vtr
2vts
2vtt
2vu3
2vv9
2w05
2w06
2w17
2w1h
2wev
2wfy
2whb
2wih
2wip
2wma
2wmb
2wpa
2wxv
2x1n
2xmy
2xnb
3bht
3bhu
3bhv
3ddp
3ddq
3dog
3eid
3ej1
3eoc
3ezr
3ezv
3f5x
3fz1
3ig7
3igg
3le6
3lfn
3lfq
3lfs
3my5


In [19]:
%ls

ad4_multiDock_PARES.py  CDK2_INHIBIDORES_Hs/  Obtener_ligandos_CDK2.ipynb
CDK2_INHIBIDORES/       envio_miztli/         prueba/


In [32]:
import pybel

mol = pybel.readfile(filename="./CDK2_INHIBIDORES/1aq1-STU.pdb", format="pdb").next()
print(mol.atoms)

AttributeError: 'generator' object has no attribute 'next'